## This is the initial network design for chromstem-net

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

This is the dataset generator class, to be loaded into the DataLoader tool

This is adapted from the pytorch tutorial: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [3]:
class ChromstemDataset(Dataset):
    """Chromstem Dataset"""
    
    def __init__(self,csv_file,root_dir,transform=None):
        """
        Args:
            csv_file (string): Master CSV file that will hold all the data
            root_dir (string): Directory where the datasets are labeled
            transform (callable, optional): Optional transformations to samples
        """
        
        f = open(csv_file,'r')
        csv_length = len(f.readlines()[-1].strip())
        f.close()
        col_names = ['File'] + ['Nucl%d' % i for i in np.arange(1,csv_length+2)]
        self.nucl_coords_frame_ = pd.read_csv(csv_file,engine='python',header=None,names=col_names)
        self.root_dir_ = root_dir
        self.transform_ = transform
    
    def read_chromstem(self,fnme):
        x,y,z,rho = np.loadtxt(fnme,comments='#',unpack=True)
        tensor = torch.zeros_like(torch.empty(int(x[0]),int(y[0]),int(z[0])))
    
        for i,arr in enumerate(zip(x[1:],y[1:],z[1:])):
            tensor[int(arr[0]),int(arr[1]),int(arr[2])] = rho[i]
        
        # Convert to sparse matrix as data will be sparse
        #tensor = torch.Sparse.FloatTensor(tensor)
        return tensor
    
    def __len__(self):
        return(len(self.nucl_coords_frame_))
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        chromstem_name = os.path.join(self.root_dir_,self.nucl_coords_frame_.iloc[idx,0])
        chromstem = self.read_chromstem(chromstem_name)
        nucl_coords = self.nucl_coords_frame_.iloc[idx,1:]
        nucl_coords = np.asarray([nucl_coords])
        nucl_coords = nucl_coords.astype('float').reshape(-1,3)
        
        sample = {'chromstem' : chromstem, 'nucl_coords' : nucl_coords}
        
        if self.transform_:
            sample = self.transform_(sample)
            
        return sample
        

Here, would be useful to provide a plotting function for samples in the dataset.
The main input to this function is any sample from the dataset and it would output the density and "centromere" in a 3D plot.
To do so accurately, I am going to use the ipyvolume widget.

In [4]:
import numpy as np
import ipyvolume as ipv

In [5]:
def plot_sample(sample):
    ipv.figure()
    # Plot voxels of density
    ipv.scatter(sample['chromstem'][:,0],sample['chromstem'][:,1],sample['chromstem'][:,2], 
                color='blue',size=10,marker='box',opacity=0.5)
    
    # Plot nucleosome centers
    ipv.scatter(sample['nucl_coords'][:,0],sample['nucl_coords'][:,1],sample['nucl_coords'][:,2],
               color='red',size=5,marker='sphere')
    
    ipv.show()
    return

This is a quick test of the widget and plotting function before the data is loaded

In [6]:
sample = {'chromstem' :  np.asarray([[1,2,3]]), 'nucl_coords' : np.asarray([[0,1,2],[5,10,0.2],[10,3.3,2]])}
plot_sample(sample)

Here is where the dataset is loaded in. The .csv file should be located in the current directory, so the root is '.'

In [7]:
chromstem_dataset = ChromstemDataset(csv_file='output_label_test.csv',
                                     root_dir='.')

Here is a quick test to make sure the dataset looks appropriate

In [8]:
print(chromstem_dataset[20])

ValueError: cannot reshape array of size 148 into shape (3)

Now I will load in the entire dataset and begin to feed it into a built neural network

In [9]:
chromstem_dataset = ChromstemDataset(csv_file='output_label.csv',
                                     root_dir='.')

To start, the dataloader will be able to shuffle the data, and I will start with a batch size of 4. No idea if that will work, but for now this is a fixed value. 

In [20]:
trainloader = DataLoader(chromstem_dataset, batch_size=1,
                        shuffle=True, num_workers=2)

testloader  = DataLoader(chromstem_dataset, batch_size=1,
                         shuffle=False, num_workers=2)

Finally, the neural network class will be initialized here.

In [18]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv3d(1,3,3,stride=1)
        self.pool  = nn.MaxPool3d(2,2)
        self.conv2 = nn.Conv3d(3,9,3)
        self.fc1   = nn.Linear(9*3*3,27)
        self.fc2   = nn.Linear(27,9)
        self.fc3   = nn.Linear(9,3)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 9*3*3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

To start, the cross entropy loss function is chosen, however a different function may be required

In [12]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

If the user has CUDA functionality on their computer, the network trains significantly faster with GPU. As such, here I will provide the opportunity to use CUDA. A correct output would say that the device being used is "cuda:0"

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

net.to(device)

cuda:0


Net(
  (conv1): Conv3d(1, 3, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(3, 9, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (fc1): Linear(in_features=81, out_features=27, bias=True)
  (fc2): Linear(in_features=27, out_features=9, bias=True)
  (fc3): Linear(in_features=9, out_features=3, bias=True)
)

Train the network briefly here

In [21]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['chromstem'].to(device), data['nucl_coords'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: Expected 5-dimensional input for 5-dimensional weight 3 1 3 3 3, but got 4-dimensional input of size [1, 34, 34, 34] instead